<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Displaying images using Firefly</b> <br>
Last verified to run on 2021-06-27 with LSST Science Pipelines release w_2021_33 <br>
Contact authors: Jeff Carlin <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

**Credit:** This tutorial is based on the `Firefly.ipynb` notebook that is available under your home directory, in "notebooks/notebook-demo/".



### Learning Objectives

This notebook is a counterpart to tuturial notebook 03_Image_Display_and_Manipulation.ipynb, and is intended to demonstrate the [Firefly](https://mospace.umsystem.edu/xmlui/handle/10355/5346) interactive interface for viewing image data. 

This tutorial seeks to teach you about how to use the LSST Science Pipelines to inspect outputs from single frame processing by displaying images and source catalogs in the Firefly image viewer. In doing so, you’ll be introduced to some of the LSST Science Pipelines’ Python APIs, including:

* Accessing datasets with the `Butler`.
* Displaying images with `lsst.afw.display`
* Pass source catalog data directly to the `FireflyClient`

### Set Up

In [1]:
# What version of the Stack are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

Recommended (Weekly 2021_33)
lsst_distrib          22.0.1-3-g7ae64ea+3baa4596b0 	current w_2021_33 setup


### 1. Load the LSST Science Pipelines

First, we load the `lsst.afw.display` library to gain access to the image visualization routines we'd like to use, and the `lsst.daf.butler` library, which is used to access data products.

In [2]:
# load lsst.afw.display to gain access to image visualization routines.
import lsst.afw.display as afwDisplay
# load the Butler, which provides programmatic access to LSST data products.
from lsst.daf.butler import Butler

### 2. Load the Data to Visualize

To display an image, we must first load some data. These data have been processed with the LSST Science Pipelines, and are organized in a structure that enables us to access them through the `Butler`. For more information on the `Butler`, see [lsst.daf.butler](https://pipelines.lsst.io/modules/lsst.daf.butler/index.html).

The DP0.1 data set contains simulated images from the LSST DESC Data Challenge 2 (DC2). These data are available in an S3 bucket `s3://butler-us-central1-dp01`. We access a single image from a specific visit (`192350`) and detector (`175`).

Once we define a string that contains the data directory, we start the `Butler` instance using the `lsst.daf.butler` library and its `Butler` class. The `Butler` object is initialized with a string containing the data directory we wish to access. Running the cell may take a few moments.

With the `Butler` instance now generated using our data directory, we can retrieve the desired calibrated exposure by telling the butler which filter ("band"), CCD ("detector", and visit we wish to view. To do this, we define a dictionary with the required information.

In [3]:
dataId = {'visit': 192350, 'detector': 175, 'band': 'i'}
# Note: because the combination of visit+detector already uniquely identifies
# the exposure, specifying "band" above is unnecessary.

repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

# Retrieve the data using the `butler` instance and its function `get()`
calexp = butler.get('calexp', **dataId)

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-0.6.0/Linux64/obs_base/22.0.1-17-gd7f467c+f517c7078c/python/lsst/obs/base/formatters/fitsExposure.py:622: UserWarning: Data ID {instrument: 'LSSTCam-imSim', detector: 175, visit: 192350} is missing (implied) value(s) for ['band', 'physical_filter']; the correctness of this Exposure's FilterLabel cannot be guaranteed. Call Registry.expandDataId before Butler.get to avoid this.
  warnings.warn(f"Data ID {self.dataId} is missing (implied) value(s) for {missing}; "


## Create a Display

To display the `calexp` you will use the LSST `afwDisplay` framework. It provides a uniform API for multiple display backends, including DS9, matplotlib, and LSST’s Firefly viewer. The default backend is `ds9`, but since we are working remotely on `JupyterLab` we would prefer to use the web-based Firefly display. A [user guide](https://pipelines.lsst.io/v/daily/modules/lsst.display.firefly/index.html)  for `lsst.display.firefly` is available on the [pipelines.lsst.io](https://pipelines.lsst.io/v/daily) site.

Now, we create a Firefly display.

In [4]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)

In the Science Platform Notebook aspect, a Firefly viewer tab appears. You may wish to drag it to the right side of the JupyterLab area to create two side by side panes, one with the notebook and one with the display.

## Display the calexp (calibrated exposure)

We can now build the display and use the `mtv` method to view the `calexp` with Firefly. First we display an image with mask planes and then overplot some sources.

In [5]:
afw_display.mtv(calexp)

As soon as you execute the command a single calibrated HSC exposure for the `{'physical_filter': 'HSC-I', 'detector': 87, 'visit': 1228}` data ID should appear in the Firefly `JupyterLab` tab.

Notice that the image is overlaid with colorful regions. These are mask regions. Each color reflects a different mask bit that correspond to detections and different types of detector artifacts. You’ll learn how to interpret these colors later, but first you’ll likely want to adjust the image display.

## Improving the image display

The display framework gives you control over the image display to help bring out image details. For example, to make masked regions semi-transparent, so that underlying image features are visible, try:

In [6]:
afw_display.setMaskTransparency(80)

The setMaskTransparency method’s argument can range from 0 (fully opaque) to 100 (fully transparent).

You can also control the colorbar scaling algorithm with the display’s scale method. Try an asinh stretch with explicit minimum (black) and maximum (white) values:

In [7]:
afw_display.scale("asinh", -5, 20)

You can also use an automatic algorithm like `zscale` (or `minmax`) to select the white and black thresholds:

In [8]:
afw_display.scale("asinh", "zscale")

## Interpreting displayed mask colors

The display framework renders each plane of the mask in a different color (plane being a different bit in the mask). To interpret these colors you can get a dictionary of mask planes from the `calexp` and query the display for the colors it rendered each mask plane with. For example:

In [9]:
mask = calexp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, afw_display.getMaskPlaneColor(maskName)))

BAD: red
CR: magenta
CROSSTALK: None
DETECTED: blue
DETECTED_NEGATIVE: cyan
EDGE: yellow
INTRP: green
NOT_DEBLENDED: None
NO_DATA: orange
SAT: green
SUSPECT: yellow
UNMASKEDNAN: None


In the Firefly viewer tab, the overlays button ![overlays button](http://irsa.ipac.caltech.edu/onlinehelp/finderchart/img/layers.png) on the toolbar gives you very detailed control over the mask planes, such as turning individual planes on and off, changing the color and adjusting the transparency. Mask transparency and colors can also be set using `afw.display` commands, for individual planes or for all.

## Plotting sources on the display

The LSST processing pipeline also creates a table of the sources it used for PSF estimation as well as astrometric and photometric calibration. The dataset type of this table is `src`, which you can get from the Butler:

In [10]:
src = butler.get('src', **dataId)

The returned object, `src`, is a `lsst.afw.table.SourceTable` object. `SourceTables` are explored more elsewhere, but you can do some simple investigations using common python functions. For example, to check the length of the object:




In [11]:
len(src)

2442

You can view an HTML rendering of the `src` table by getting an `astropy.table.Table` version of it:

In [12]:
src.asAstropy()

id,coord_ra,coord_dec,parent,calib_detected,calib_psf_candidate,calib_psf_used,calib_psf_reserved,deblend_nChild,deblend_deblendedAsPsf,deblend_psfCenter_x,deblend_psfCenter_y,deblend_psf_instFlux,deblend_tooManyPeaks,deblend_parentTooBig,deblend_masked,deblend_skipped,deblend_rampedTemplate,deblend_patchedTemplate,deblend_hasStrayFlux,base_NaiveCentroid_x,base_NaiveCentroid_y,base_NaiveCentroid_flag,base_NaiveCentroid_flag_noCounts,base_NaiveCentroid_flag_edge,base_NaiveCentroid_flag_resetToPeak,base_SdssCentroid_x,slot_Centroid_x,base_SdssCentroid_y,slot_Centroid_y,base_SdssCentroid_xErr,slot_Centroid_xErr,base_SdssCentroid_yErr,slot_Centroid_yErr,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_GaussianFlux_flag_badCentroid,base_LocalBackground_flag_badCentroid,base_NaiveCentroid_flag_badInitialCentroid,base_PsfFlux_flag_badCentroid,base_SdssShape_flag_badCentroid,base_Variance_flag_badCentroid,ext_photometryKron_KronFlux_flag_badInitialCentroid,ext_shapeHSM_HsmPsfMoments_flag_badCentroid,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid,ext_shapeHSM_HsmSourceMoments_flag_badCentroid,slot_Centroid_flag,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_GaussianFlux_flag_badCentroid_edge,base_LocalBackground_flag_badCentroid_edge,base_NaiveCentroid_flag_badInitialCentroid_edge,base_PsfFlux_flag_badCentroid_edge,base_SdssShape_flag_badCentroid_edge,base_Variance_flag_badCentroid_edge,ext_photometryKron_KronFlux_flag_badInitialCentroid_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_edge,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_edge,slot_Centroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_GaussianFlux_flag_badCentroid_noSecondDerivative,base_LocalBackground_flag_badCentroid_noSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,base_SdssShape_flag_badCentroid_noSecondDerivative,base_Variance_flag_badCentroid_noSecondDerivative,ext_photometryKron_KronFlux_flag_badInitialCentroid_noSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_noSecondDerivative,slot_Centroid_flag_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_GaussianFlux_flag_badCentroid_almostNoSecondDerivative,base_LocalBackground_flag_badCentroid_almostNoSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,base_SdssShape_flag_badCentroid_almostNoSecondDerivative,base_Variance_flag_badCentroid_almostNoSecondDerivative,ext_photometryKron_KronFlux_flag_badInitialCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_almostNoSecondDerivative,slot_Centroid_flag_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_GaussianFlux_flag_badCentroid_notAtMaximum,base_LocalBackground_flag_badCentroid_notAtMaximum,base_NaiveCentroid_flag_badInitialCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,base_SdssShape_flag_badCentroid_notAtMaximum,base_Variance_flag_badCentroid_notAtMaximum,ext_photometryKron_KronFlux_flag_badInitialCentroid_notAtMaximum,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_notAtMaximum,ext_shapeHSM_Hs

Now we’ll overplot sources from the `src` table onto the image display using the Display’s `dot` method for plotting markers. `Display.dot` plots markers individually, so you’ll need to iterate over rows in the `SourceTable`.  It is more efficient to send a batch of updates to the display, so we enclose the loop in a `display.Buffering` context, like this:

In [13]:
with afw_display.Buffering():
    for record in src:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

## Clearing markers

`Display.dot` always adds new markers to the display. To clear the display of all markers, use the erase method:

In [14]:
afw_display.erase()

In [15]:
dataId_coadd = {'tract': 4431, 'patch': 17, 'band': 'i'}

# Retrieve the data using the `butler` instance and its function `get()`
coadd_calexp = butler.get('deepCoadd_calexp', **dataId_coadd)

In [16]:
forced_src = butler.get('deepCoadd_forced_src', **dataId_coadd)

In [17]:
afw_display.mtv(coadd_calexp)

In [ ]:
### Add some quality cuts before the next cell so it's not plotting "bad" sources?

In [ ]:
with afw_display.Buffering():
    for record in forced_src:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

In [ ]:
afw_display.erase()

## Wrap up
In this tutorial we have used the `Butler` to access LSST simulation data and used the LSST Science Pipelines Python API to display images and tables. Here are some key takeaways:

* Use the `lsst.daf.persistence.Butler` class to read and write data from repositories.
* The `lsst.afw.display` module provides a flexible framework for sending data from LSST Science Pipelines code to image displays. We used the Firefly backend for web-based visualization (`ds9`, `matplotlib`, and `ginga` are other avialable backends).
* `Exposure` objects have image data, mask data, and metadata. When you display an Exposure, the display framework automatically overlays mask planes.
* We have accessed and visualized the Table of sources extracted from an image. The `Table.asAstropy` method can be used to view the table as an `astropy.table.Table`.
* We have sent a subset of objects to directly to the Firefly API for additional plotting and investigation.


